<a href="https://colab.research.google.com/github/ByungjunKim/EnergyTransitionKorea/blob/main/EnergyTransitionKorea_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding the public’s attitudes towards energy and environmental issues using text data

실습용 코드 (CatBoost 모델링 제외한 각종 데이터 전처리와 분류 모델 써보기)  

In [ ]:
# 필요한 패키지 설치
!pip install -q transformers lightning torch gdown kiwipiepy

In [ ]:
# 필요한 파이썬 패키지 로드
import numpy as np

from tqdm.notebook import tqdm

import torch ## version >= 1.8.2
import torch.nn as nn

import pytorch_lightning as pl ## version == 1.9.0
from pytorch_lightning import Trainer

from transformers import AutoTokenizer, AutoModel ## version == 4.12.3
import os

import torch
from torch.optim.lr_scheduler import ExponentialLR
from pytorch_lightning import LightningModule, Trainer, seed_everything
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import Trainer

import pandas as pd
import polars as pls
import json
from tqdm import tqdm
from json import JSONDecodeError
import pickle

tqdm.pandas()
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 200 # DPI 고화질로 향상
# plt.rcParams['font.family'] = 'NanumGothic'
import seaborn as sns

# from mpire import WorkerPool
# import multiprocessing
# n_cpu = multiprocessing.cpu_count()
from itertools import chain
import glob
from natsort import natsorted
from scipy import stats
import gdown

from kiwipiepy import Kiwi
kiwi = Kiwi(model_type='sbg')

from collections import Counter
from itertools import chain

# Gensim
import gensim
import gensim.corpora as corpora
from gensim import corpora, models
from gensim.utils import simple_preprocess

from transformers import AutoTokenizer, BertForSequenceClassification, logging
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer
logging.set_verbosity_error()
import re
# import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings(action='ignore')
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats import kruskal

In [ ]:
# Clone
!git clone https://github.com/ByungjunKim/EnergyTransitionKorea.git

### Data Load

##### news

In [ ]:
news = pd.read_excel('./EnergyTransitionKorea/data/news_df.xlsx')
news

In [ ]:
# 연구에 활용할 뉴스 기사만 선별하기 (inclusion 컬럼 활용)
news = news.loc[news['inclusion']==1].reset_index(drop=True)
news

In [ ]:
# 제목의 긍정(1), 중립(0), 부정(-1) 분류 (복수의 연구자들의 qualitative한 분류)
news['title_p/n'].value_counts(normalize=True)

In [ ]:
# 에너지 정책에 대한 논조 긍정(1), 중립(0), 부정(-1) 분류 (복수의 연구자들의 qualitative한 분류)
news['e_policy_p/n'].value_counts(normalize=True)

In [ ]:
# User Profile
# Reading the CSV file into a DataFrame
user_profile_df = pd.read_csv('./EnergyTransitionKorea/data/user_profile_df.csv')
user_profile_df

In [ ]:
# 댓글 데이터
reply = pd.read_csv('./EnergyTransitionKorea/data/reply_df_catboost.csv')
reply

In [ ]:
# 컬럼 확인
reply.columns

### Preprocess
파생변수(Derived Variable) 만들기

##### Tokenization for TTR

In [ ]:
def tokenize(sent):
    res = kiwi.tokenize(sent, normalize_coda=True)
    return [word+'/'+tag for word, tag, _, _ in res
            if not tag.startswith('W')] # 웹관련(w) 형태소 제거

In [ ]:
# 토크나이징(한국어 형태소 분석) 테스트
kiwi.tokenize('안녕하세요 KAIST 넙죽이입니다. sdfkljskf@kaist.ac.kr')

In [ ]:
# 토크나이징(한국어 형태소 분석) 테스트
tokens = tokenize('안녕하세요 KAIST 넙죽이입니다. 넙죽넙죽 sdfkljskf@kaist.ac.kr')
tokens

In [ ]:
# TTR(Type Token Ratio) 계산
def calculate_ttr(tokens):
    # Tokenize the text into words
    # tokens = text.split()

    # Count the total number of tokens
    total_tokens = len(tokens)

    if total_tokens!=0:
        # Count the number of unique tokens (types)
        unique_tokens = len(set(tokens))

        # Calculate the Term-to-Term Ratio (TTR)
        ttr = unique_tokens / total_tokens
    else:
        ttr = 0

    return ttr

In [ ]:
calculate_ttr(tokens)

In [ ]:
# 5분 내외 걸림
reply['tokens'] = reply['reply'].progress_map(lambda x:tokenize(x))

# with WorkerPool(n_jobs=n_cpu) as pool:
#     tokens_list = pool.map(tokenize, reply['reply'].tolist(), progress_bar=True)

In [ ]:
reply['tokens']

##### 상위 명사 추출하기

In [ ]:
# 명사만 추출 (NNG=보통명사, NNP=공유명사)
nouns = reply['tokens'].tolist()
nouns = [[w for w in l if w.split('/')[1]=='NNG' or w.split('/')[1]=='NNP'] for l in nouns]

In [ ]:
# 상위 n개 Unigram 확인
nouns_tokens = chain(*nouns)
cnt = Counter(nouns_tokens)
pd.DataFrame(cnt.most_common(20), columns=['Unigram','Frequency'])

In [ ]:
def make_bigram(data_words,bigram_min_count):
    bigram = gensim.models.Phrases(data_words, min_count=bigram_min_count)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    # bigram으로 만들기
    data_words_bigrams = [bigram_mod[doc] for doc in data_words] # 불용어 1차 제거 후에 bigram
    return data_words_bigrams

In [ ]:
bigram = make_bigram(nouns,100)  # N회 이상 등장한 단어만 bigram 처리

In [ ]:
# bigram 단어 확인
bigram_list = [word for sent in bigram for word in sent if "_" in word]
pd.DataFrame(Counter(bigram_list).most_common(20), columns=['Bigram','Frequency']).to_excel('./bigram_keywords.xlsx')
Counter(bigram_list).most_common(20) # 상위 N개

##### User Clustering (K-means)

In [ ]:
# Convert the 'start_date' column to datetime format
user_profile_df['start_date'] = pd.to_datetime(user_profile_df['start_date'])

# Calculate the number of days since the start date
current_date = datetime.now()
user_profile_df['days_since_start'] = (current_date - user_profile_df['start_date']).dt.days

# Display the first few rows of the DataFrame with the new column
user_profile_df.head()

In [ ]:
# Remove commas from the columns and convert them to integers
columns_to_convert = ['sum_reply', 'sum_son_reply', 'recent_n_reply']

for col in columns_to_convert:
    user_profile_df[col] = user_profile_df[col].str.replace(',', '').astype(int)

# Check the data types and first few rows to ensure the columns are converted correctly
user_profile_df.dtypes, user_profile_df.head()

In [ ]:
# Select the features for clustering
selected_features = ['sum_reply', 'sum_son_reply', 'recent_n_reply', 'days_since_start']
cluster_data = user_profile_df[selected_features]

# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(cluster_data)

# Apply K-means clustering (k=2 for "heavy" and "normal")
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(scaled_data)

# Add the cluster labels to the original DataFrame
user_profile_df['user_type'] = np.where(kmeans.labels_ == 0, 'normal', 'heavy')

# Display first few rows with the new 'user_type' column
user_profile_df.head()

### Classification Model

##### News Article Classification (한국언론진행재단에서 개발한 BERT 기반 분류 모델)
https://github.com/KPF-bigkinds/BIGKINDS-LAB/tree/main/KPF-BERT-CLS

In [ ]:
# KPFBERT(한국언론진흥재단 BERT 가져오기)
!git clone https://github.com/KPFBERT/kpfbert.git

In [ ]:
big_label = ['정치', '경제', '사회', '문화', '국제', '스포츠', 'IT_과학']
big_label2id = {label: i for i, label in enumerate(big_label)}
big_id2label = {i: label for label, i in big_label2id.items()}

In [ ]:
big_id2label

In [ ]:
# tokenizer 및 model 불러오기
kpf_tokenizer = AutoTokenizer.from_pretrained("kpfbert")
# huggingface 분류모델 불러오기
kpf_model1 = BertForSequenceClassification.from_pretrained("KPF/KPF-bert-cls2")

In [ ]:
pipe = TextClassificationPipeline(
        model = kpf_model1,
        tokenizer = kpf_tokenizer,
        device = 0, # gpu: 0
        top_k=None,
        function_to_apply = 'sigmoid')

In [ ]:
# 신문 기사 제목 분류 연습
sorted(pipe('''여소야대 높은 벽에 ‘부동산稅’ 빠진 세법 개정안… 세수효과 -4719억원''')[0], key=lambda d: d['label'])

##### Emotion Classification
https://github.com/kjhkjh95/Moral_Emotion_Dataset  
김재홍, 정채윤, 차미영, 이원재. (2023). KOME(Korean Online Moral Emotion): 세부 분류를 위한 한국어 도덕감정 데이터셋. 한국정보과학회 학술발표논문집.
https://www.dbpia.co.kr/pdf/pdfView.do?nodeId=NODE11488020

In [ ]:
electra = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022", return_dict=True)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
LABELS = ['Neutral', 'Non-Moral-Emotion', 'Other-Condemming', 'Other-Praising', 'Other-Suffering', 'Self-Conscious']

In [ ]:
# https://drive.google.com/file/d/1TWQek8ouPLrunIFvB-aOLbbS6MO2iozc/view?usp=sharing
# KOME 체크포인트 파일 다운로드
gdown.download(id='1TWQek8ouPLrunIFvB-aOLbbS6MO2iozc',output='best_model.ckpt')

In [ ]:
electra = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022", return_dict=True)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
LABELS = ['Neutral', 'Non-Moral-Emotion', 'Other-Condemming', 'Other-Praising', 'Other-Suffering', 'Self-Conscious']


INITIAL_LR = 2e-5
class KOTETagger(pl.LightningModule):

    def __init__(self, n_training_steps=None, n_warmup_steps=None, gamma_for_expLR=None):
        super().__init__()
        self.electra = electra
        self.classifier = nn.Linear(self.electra.config.hidden_size, 6) ## the label dimension == 6 <-- what an ominous number for asians though... <-- I didn't intend it!
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps

        ## the loss
        self.criterion = nn.BCELoss()

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.electra(input_ids, attention_mask=attention_mask)
        output = output.last_hidden_state[:,0,:] ## [CLS] of the last hidden state
        output = self.classifier(output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)

        torch.cuda.empty_cache()

        return loss, output

    def step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self.forward(input_ids, attention_mask, labels)

        preds = outputs

        y_true = list(labels.detach().cpu())
        y_pred = list(preds.detach().cpu())

        return {"loss": loss, "y_true": y_true, "y_pred": y_pred}

    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx)

    def epoch_end(self, outputs, state="train"):
        loss = torch.tensor(0, dtype=torch.float)
        for out in outputs:
            loss += out["loss"].detach().cpu()
        loss = loss / len(outputs)

        y_true = []
        y_pred = []
        for out in outputs:
            y_true += out["y_true"]
            y_pred += out["y_pred"]

        self.log(state + "_loss", float(loss), on_epoch=True, prog_bar=True)
        print(f"[Epoch {self.trainer.current_epoch} {state.upper()}] Loss: {loss}")
        return {"loss": loss}

    def training_epoch_end(self, outputs):
        self.epoch_end(outputs, state="train")

    def validation_epoch_end(self, outputs):
        self.epoch_end(outputs, state="val")

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=INITIAL_LR, weight_decay=0.01)

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )

        return dict(
          optimizer=optimizer,
          lr_scheduler=dict(
            scheduler=scheduler,
            interval="step"
          )
        )

In [ ]:
from glob import glob

best_ckpt = "./best_model.ckpt"

moral_emotion_detection = KOTETagger.load_from_checkpoint(best_ckpt, map_location=torch.device('cuda'),strict=False)
moral_emotion_detection.eval()
moral_emotion_detection.freeze()
DEVICE = torch.device(f"cuda:0")  # GPU 번호 설정
moral_emotion_detection = moral_emotion_detection.to(DEVICE)

In [ ]:
THRESHOLD = 0.8  # 예를 들어 0.8로 지정. 원하는 값으로 조절 가능.

sample_text = "아이고 저 사람들은 불쌍해서 어떡하냐 ㅠㅠ" # 분류하고 싶은 문장을 넣어보세요
encoding = tokenizer.encode_plus(
  sample_text,
  add_special_tokens=True,
  max_length=512,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors="pt",
)

_, predictions = moral_emotion_detection(encoding["input_ids"].to(DEVICE), encoding["attention_mask"].to(DEVICE))
predictions = predictions.flatten().cpu()
for l, p in zip(LABELS, predictions):
    if p < THRESHOLD:
        continue
    print(f"{l}: {p}")

### EDA

##### User Clustering

In [ ]:
user_profile_df['user_type'].value_counts(normalize=True)

In [ ]:
# Summary statistics for each user type
selected_features = ['sum_reply', 'sum_son_reply', 'recent_n_reply', 'days_since_start']
summary_stats = user_profile_df.groupby('user_type')[selected_features].describe()

# Visualizations
plt.figure(figsize=(15, 10))

for i, feature in enumerate(selected_features, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(x='user_type', y=feature, data=user_profile_df)
    plt.title(f'Boxplot of {feature} by User Type')

plt.tight_layout()
plt.show()

In [ ]:
summary_stats['days_since_start']

In [ ]:
# Visualizations
plt.figure(figsize=(15, 10))

for i, feature in enumerate(selected_features, 1):
    plt.subplot(2, 2, i)
    # Q1, Q3, 그리고 IQR 계산
    Q1 = user_profile_df[feature].quantile(0.25)
    Q3 = user_profile_df[feature].quantile(0.75)
    IQR = Q3 - Q1

    # IQR을 사용하여 이상치 찾기 및 제거
    user_profile_df_ = user_profile_df[(user_profile_df[feature] >= Q1 - 1.5 * IQR) & (user_profile_df[feature] <= Q3 + 1.5 * IQR)]
    sns.boxplot(x='user_type', y=feature, data=user_profile_df_)
    plt.title(f'Boxplot of {feature} by User Type (IQR)')

plt.tight_layout()
plt.show()

##### News Article Classification

In [ ]:
reply.columns

In [ ]:
big_label = ['politics', 'economy', 'society', 'culture', 'international', 'sports', 'IT_science']
reply[big_label].describe()

In [ ]:
g = sns.boxplot(data = reply[big_label])
g.set_xticklabels(g.get_xticklabels(), rotation=45)

##### Emotion Classification

In [ ]:
LABELS = ['Condemning','Praising', 'Suffering', 'Self-Conscious']
reply[LABELS].describe()

In [ ]:
g = sns.boxplot(data = reply[LABELS])
g.set_xticklabels(g.get_xticklabels(), rotation=45)

##### Kruskal-Wallis H test (2개 이상의 집단의 평균 차이 검정)
https://en.wikipedia.org/wiki/Kruskal%E2%80%93Wallis_one-way_analysis_of_variance

In [ ]:
# Define a function to perform the Kruskal-Wallis H test for the dependent variable based on the specified group column
def perform_kruskal_test_single(df, group_column, dependent_var):
    groups = [df[dependent_var][df[group_column] == group].values for group in df[group_column].unique() if group == group]  # Exclude NaN groups
    h_stat, p_val = kruskal(*groups)
    return h_stat, p_val

# Perform the Kruskal-Wallis H test based on 'user_type', 'title_p/n', and 'e_policy_p/n' for 'Condemning' and store the results
kruskal_results_single = {
    'user_type': perform_kruskal_test_single(reply, 'user_type', 'Condemning'),
    'title_p/n': perform_kruskal_test_single(reply, 'title_p/n', 'Condemning'),
    'e_policy_p/n': perform_kruskal_test_single(reply, 'e_policy_p/n', 'Condemning')
}

kruskal_results_single

In [ ]:
reply.groupby('title_p/n')['Condemning'].describe()

In [ ]:
reply.boxplot(by='title_p/n', column='Condemning', grid=False)